In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score ## To compute the auc score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

## FOR NORMALIZATION AND PREPROCESSING

In [97]:
def drop_columns(data):
    drop_columns = []
    for column in data.columns:
        if column != 'target' and data[column].nunique() < 2:
            drop_columns.append(column)
        elif (data[column] == -1).sum() / data.shape[0] > 0.8:
            drop_columns.append(column)
    return data.drop(columns=drop_columns), drop_columns

        
# To be performed after drop columns is called!!!
def one_hot_encoding(data, categoricals):
    categorical_idx = []
    count = 0
    for column in categoricals:
        #if data[column].nunique() < 10 and (data[column] < 0).sum() == 0:
        
        if column in data and data[column].nunique() < 10  and (data[column] < 0).sum() == 0:
            count += 1
            #print(data.count_vals)
            categorical_idx.append(data.columns.get_loc(column))
    print(count)
    enc = OneHotEncoder(categorical_features=categorical_idx)
    return enc.fit_transform(data).toarray()
    

In [98]:
data = pd.read_csv('train_2008.csv')
data, drop_columns = drop_columns(data)
data = data.drop(columns=['id'])
y_train = data.pop('target').values

## Add one to each of the values in the dataframe in order to get rid of -1 values
data += 1

categoricals =['HURESPLI', 'HUFINAL', 'HUSPNISH', 'HETENURE', 'HEHOUSUT', \
               'HETELHHD', 'HETELAVL', 'HEPHONEO', 'HUTYPEA', 'HUTYPB', 'HUTYPC', \
               'HRINTSTA', 'HRHTYPE', 'HUINTTYP', 'HRLONGLK', 'HUBUS', 'GEREG', \
               'GESTCEN', 'GESTFIPS', 'GTCBSA', 'GTCO', 'GTCBSAST', 'GTMETSTA', \
               'PUPELIG', 'PERRP', 'PRTFAGE', 'PEMARITL', 'PESEX', 'PEAFEVER', \
               'PEAFNOW', 'PEEDUCA', 'PTDTRACE', 'PRDTHSP', 'PUCHINHH', 'PRFAMNUM', \
               'PRFAMREL', 'PRFAMTYP', 'PEHSPNON', 'PRMARSTA', 'PRPERTYP', \
               'PENATVTY', 'PEMNTVTY', 'PEFNTVTY', 'PRCITSHP', 'PUSLFPRX', \
               'PEMLR', 'PUWK', 'PUBUS1', 'PUBUS2OT', 'PUBUSCK1', 'PUBUSCK2', \
               'PUBUSCK3', 'PUBUSCK4', 'PURETOT', 'PUDIS', 'PERET1', 'PUDIS1', \
               'PUDIS2', 'PUABSOT', 'PULAY', 'PEABSRSN', 'PEABSPDO', 'PEMJOT', \
               'PEHRFTPT', 'PEHRUSLT', 'PEHRWANT', 'PEHRRSN1', 'PEHRRSN2', 'PEHRRSN3', \
               'PUHROFF1', 'PUHROT1', 'PEHRAVL', 'PULAYDT', 'PULAY6M', 'PELAYAVL', \
               'PULAYAVR', 'PELAYLK', 'PELAYFTO', 'PULK', 'PELKM1', 'PULKM2', \
               'PULKM3', 'PULKM4', 'PULKM5', 'PULKM6', 'PULKDK1', 'PULKDK2', \
               'PULKPS1', 'PULKPS2', 'PULKPS3', 'PELKAVL', 'PULKAVR', 'PELKLL1O', \
               'PELKLL2O', 'PELKLWO', 'PELKFTO', 'PEDWWNTO', 'PEDWRSN', 'PEDWLKO', \
               'PEDWWK', 'PEDW4WK', 'PEDWLKWK', 'PEDWAVL', 'PEDWAVR', 'PUDWCK1', 'PUJHDP1O', 'PEJHRSN', 'PEJHWANT', 'PRABSREA', 'PRCIVLF', 'PRDISC', 'PREMPHRS', 'PREMPNOT', 'PREXPLF', 'PRFTLF', 'PRJOBSEA', 'PRPTREA', 'PRUNTYPE', 'PRWKSCH', 'PRWKSTAT', 'PRWNTJOB', 'PUJHCK3', 'PUJHCK3', 'PUJHCK4', 'PUJHCK5', 'PUIODP1', 'PUIODP2', 'PUIODP3', 'PEIO1COW', 'PUIO1MFG', 'PEIO2COW', 'PUIO2MFG', 'PUIOCK1', 'PUIOCK2', 'PRIOELG', 'PRAGNA', 'PRCOW1', 'PRCOW2', 'PRCOWPG', 'PRDTCOW1', 'PRDTCOW2', 'PRDTIND1', 'PRDTIND2', 'PRDTOCC1', 'PRDTOCC2', 'PREMP', 'PRMJIND1', 'PRMJIND2', 'PRMJOCC1', 'PRMJOCC2', 'PRMJOCGR', 'PRNAGPWS', 'PRNAGWS', 'PRSJMJ', 'PRERELG', 'PEERNUOT', 'PEERNPER', 'PEERNRT', 'PEERNHRY', 'PEERNLAB', 'PEERNCOV', 'PENLFJH', 'PENLFRET', 'PENLFACT', 'PESCHENR', 'PESCHFT', 'PESCHLVL', 'PRNLFSCH', 'PRWERNAL', 'PRHERNAL', 'HXTENURE', 'HXHOUSUT', 'HXHOUSUT', 'HXTELAVL', 'HXPHONEO', 'PXINUSYR', 'PXRRP', 'PXPARENT', 'PXAGE', 'PXMARITL', 'PXSPOUSE', 'PXSEX', 'PXAFWHN1', 'PXAFNOW', 'PXEDUCA', 'PXRACE1', 'PXNATVTY', 'PXMNTVTY', 'PXFNTVTY', 'PXHSPNON', 'PXMLR', 'PXRET1', 'PXABSRSN', 'PXABSPDO', 'PXMJOT', 'PXMJNUM', 'PXMJNUM', 'PXHRUSL2', 'PXHRFTPT', 'PXHRUSLT', 'PXHRWANT', 'PXHRRSN1', 'PXHRRSN2', 'PXHRACT1', 'PXHRACT2', 'PXHRACTT', 'PXHRRSN3', 'PXHRAVL', 'PXLAYAVL', 'PXLAYLK', 'PXLAYDUR', 'PXLAYFTO', 'PXLKM1', 'PXLKAVL', 'PXLKLL1O', 'PXLKLL2O', 'PXLKLWO', 'PXLKDUR', 'PXLKFTO', 'PXDWWNTO', 'PXDWRSN', 'PXDWLKO', 'PXDWWK', 'PXDW4WK', 'PXDWLKWK', 'PXDWAVL', 'PXDWAVR', 'PXJHWKO', 'PXJHRSN', 'PXJHWANT', 'PXIO1COW', 'PXIO1ICD', 'PXIO1OCD', 'PXIO2COW', 'PXIO2ICD', 'PXIO2OCD', 'PXERNUOT', 'PXERNPER', 'PXERNH1O', 'PXERNWKP', 'PXERNRT', 'PXERNHRY', 'PXERNH2', 'PXERNLAB', 'PXERNCOV', 'PXNLFJH', 'PXNLFRET', 'PXNLFACT', 'PXSCHENR', 'PXSCHFT', 'PXSCHFT', 'PEDIPGED', 'PEHGCOMP', 'PEHGCOMP', 'PEGR6COR', 'PEMS123', 'PXDIPGED', 'PXHGCOMP', 'PXCYC', 'PXGRPROF', 'PXGR6COR', 'PXMS123', 'PEIO1ICD', 'PEIO2ICD', 'PEIO2OCD', 'PRIMIND1', 'PRIMIND2', 'PEAFWHN1', 'PEAFWHN2', 'PEAFWHN3', 'PEAFWHN4', 'PXAFEVER', 'PELNDAD', 'PEDADTYP', 'PEMOMTYP', 'PXLNDAD', 'PXLNMOM', 'PXDADTYP', 'PXMOMTYP', 'PXCOHAB', 'PEDISEAR', 'PEDISEYE', 'PEDISREM', 'PEDISPHY', 'PEDISDRS', 'PEDISOUT', 'PRDISFLG', 'PXDISEAR', 'PXDISEYE', 'PXDISREM', 'PXDISPHY', 'PXDISDRS', 'PXDISOUT', 'PES1', 'PES2', 'PES3', 'PES4', 'PES5', 'PES6', 'PES7', 'PUSCK4']

In [99]:
sm_results = pd.read_csv('smart_mapping_results.csv')
idx = list(sm_results['Feature']).index('HUFINAL')
drop_cols_sm = list(sm_results['Feature'])[idx:]
#data.drop(columns=drop_cols_sm, inplace=True)

In [100]:
data.shape

(64667, 192)

In [101]:
data_arr = one_hot_encoding(data, categoricals)
#X_train = preprocessing.scale(data_arr)
X_train = data_arr
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

real_data = pd.read_csv('test_2008.csv')
real_data = real_data.drop(columns=drop_columns)
real_data = real_data.drop(columns=['id'])
X_real = preprocessing.scale(np.array(real_data))

111


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/

In [102]:
data_arr.shape

(64667, 602)

In [103]:
clf_rf = RandomForestClassifier(n_estimators=100, criterion='entropy')

#clf_rf.fit(X_train, y_train)

print("cv auc score: ", np.mean(cross_val(clf_rf, X_train, y_train)['test_score']))

cv auc score:  0.7717545950119113


## Try a logistic regression model

In [21]:
from sklearn.linear_model import LogisticRegression


In [6]:
clf_lr = LogisticRegression(C=100)
clf_lr.fit(X_train, y_train)
print("AUC score is", roc_auc_score(y_test, clf_lr.predict_proba(X_test)[:, 1]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC score is 0.7715062698014148


## Add cross validation

In [43]:
## FOR NORMALIZATION

#y_train = data.pop('target').values
#X_train = preprocessing.scale(np.array(data))

In [ ]:
def drop_columns(data):
    drop_columns = []
    for column in data.columns:
        if column != 'target' and data[column].nunique() <= 2:
            drop_columns.append(column)
    return data.drop(columns=drop_columns)
data = pd.read_csv('train_2008.csv')
data = drop_columns(data)


In [ ]:
data.columns

In [46]:
y_train = data.pop('target').values
X_train = preprocessing.scale(np.array(data))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [95]:
from sklearn import model_selection

def cross_val(clf, X_train, y_train):
    
    result = model_selection.cross_validate(clf, X_train, y_train, scoring='roc_auc', cv=5, return_train_score=True)
    return result

In [12]:

#print("Random Froest cv auc score: ", np.mean(cross_val(clf_rf, X_train, y_train)['test_score']))

Decision Tree cv auc score:  0.593035023907719
Random Froest cv auc score:  0.7759626773614076


In [23]:
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.naive_bayes import GaussianNB 
clf_rf = RandomForestClassifier(n_estimators=100, criterion='entropy')
clf_lr = LogisticRegression(C=100)
clf_nb = GaussianNB()

eclf = EnsembleVoteClassifier(clfs=[clf_rf, clf_nb], weights=[1,0.1], voting='soft')
print("Ensemble cv auc score: ", np.mean(cross_val(eclf, X_train, y_train)['test_score']))

Ensemble cv auc score:  0.7693638992220867


In [18]:
from sklearn.model_selection import GridSearchCV
params = {'logisticregression__C': [_ for _ in range(20, 100, 20)],
          'randomforestclassifier__n_estimators': [20, 200, 50],}

grid = GridSearchCV(estimator=eclf, scoring='roc_auc', param_grid=params, cv=5)
grid.fit(X_train, y_train)
print("Best cv auc score is", grid.best_score_)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

Best cv auc score is 0.7455453991757339


In [21]:
grid.best_params_

{'logisticregression__C': 60, 'randomforestclassifier__n_estimators': 200}

In [24]:
params = {'n_estimators': [100, 500, 100],}
clf_rf = RandomForestClassifier()
grid = GridSearchCV(estimator=clf_rf, scoring='roc_auc', param_grid=params, cv=5)
grid.fit(X_train, y_train)
print("Best cv auc score is", grid.best_score_)

Best cv auc score is 0.7757669090551392


In [26]:
grid.best_params_

{'n_estimators': 500}

In [2]:
params = {'subsample': [0.2, 1.0, 0.4],}
gbm = GradientBoostingClassifier()
grid = GridSearchCV(estimator=gbm, scoring='roc_auc', param_grid=params, cv=5)
grid.fit(X_train, y_train)
print("Best cv auc score is", grid.best_score_)

NameError: name 'X_train' is not defined

In [34]:
grid.best_params_

{'subsample': 1.0}

In [64]:
clf_rf = RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1, max_features=30, min_samples_split=10)
gbm = GradientBoostingClassifier(subsample=1.0, n_estimators=125, min_samples_split=10, max_depth=2)

eclf = VotingClassifier(estimators=[('rf', clf_rf), ('gbm', gbm)], n_jobs=-1, voting='soft')
print("Ensemble cv auc score: ", np.mean(cross_val(eclf, X_train, y_train)['test_score']))

eclf.fit(X_train, y_train)


Ensemble cv auc score:  0.784480476473609


VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=30, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_we...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=None)

In [65]:
#print("Ensemble cv auc score: ", np.mean(cross_val(eclf, X_train, y_train)['test_score']))
eclf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=30, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_we...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=None)

In [47]:
gbm_norm = GradientBoostingClassifier(subsample=1.0, n_estimators=125, min_samples_split=10, max_depth=2)
print("Ensemble cv auc score: ", np.mean(cross_val(gbm_norm, X_train, y_train)['test_score']))
#gbm_norm.fit(X_train, y_train)


Ensemble cv auc score:  0.7771510867935303


In [62]:
def process_for_submission(y_pred, title):
    lst = y_pred
    
    print(len(lst))
    
    indexes = [i for i in range(len(lst))]
    result = []
    result.append(indexes)
    result.append(lst)
    result = np.array(result)
    result = np.transpose(result)

    
    
    #df = df.astype(int)
    df = pd.DataFrame(result, columns=['id', 'target'])
    df["id"] = df["id"].astype("int")
    df["target"] = df["target"].astype("float")
    df = df.to_csv(title + 'submission.csv', index = None, header=True)

    
    return df
    

In [13]:
process_for_submission(gbm_norm.predict_proba(X_real), 'gbm')

16000
16000


In [66]:
process_for_submission(eclf.predict_proba(X_real)[:, 1], 'eclf_rf_gbm')

16000


In [70]:
data['id']

0        16000
1        16001
2        16002
3        16003
4        16004
5        16005
6        16006
7        16007
8        16008
9        16009
10       16010
11       16011
12       16012
13       16013
14       16014
15       16015
16       16016
17       16017
18       16018
19       16019
20       16020
21       16021
22       16022
23       16023
24       16024
25       16025
26       16026
27       16027
28       16028
29       16029
         ...  
64637    80637
64638    80638
64639    80639
64640    80640
64641    80641
64642    80642
64643    80643
64644    80644
64645    80645
64646    80646
64647    80647
64648    80648
64649    80649
64650    80650
64651    80651
64652    80652
64653    80653
64654    80654
64655    80655
64656    80656
64657    80657
64658    80658
64659    80659
64660    80660
64661    80661
64662    80662
64663    80663
64664    80664
64665    80665
64666    80666
Name: id, Length: 64667, dtype: int64